In [12]:
# !pip install torch
# print(torch.__version__)
# !pip install voila
# !jupyter serverextension enable --sys-prefix voila

In [2]:
from fastai.vision.all import *
from fastai.vision.widgets import *
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [3]:
#model = torch.load('facemask.pt')
#model.cpu()
#model.eval()
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [4]:
def iou(box1, box2):
    b1_area = (box1[2]-box1[0]) * (box1[3]-box1[1])
    b2_area = (box2[2]-box2[0]) * (box2[3]-box2[1])
    inter_x1 = max(box1[0], box2[0])
    inter_x2 = min(box1[2], box2[2])
    inter_y1 = max(box1[1], box2[1])
    inter_y2 = min(box1[3], box2[3])
    inter_area = max(0, (inter_x2 - inter_x1) * (inter_y2 - inter_y1))
    union_area = b1_area + b2_area - inter_area
    iou = inter_area / union_area
    return(iou)

In [5]:
def remove_dupe_boxes(pred, thresh=0.75):
    new_boxes = []
    new_labels = []
    new_scores = []
    
    for i in range(0,len(pred['boxes'])):
        this_box = pred['boxes'][i]
        dupe = False
        for that_box in new_boxes:
            this_iou = iou(this_box, that_box)
            if this_iou > thresh:
                dupe = True
                break
        if dupe == False:
            new_boxes.append(this_box.tolist())
            new_labels.append(int(pred['labels'][i]))
            new_scores.append(float(pred['scores'][i]))
    
    new_pred = {'boxes': torch.Tensor(new_boxes), 'labels': torch.IntTensor(new_labels), 'scores': torch.Tensor(new_scores)}
    return new_pred

In [6]:
def plot_pred(idx, min_score=0.5, iou_thresh=0.75):
    img, anno = dataset.__getitem__(idx)
    img = img.to(device)

    with torch.no_grad():
        pred = model(img.unsqueeze(0))[0]
    
    pred = remove_dupe_boxes(pred, iou_thresh)
    bool_mask = pred['scores'] > min_score
    pred['boxes'] = pred['boxes'][bool_mask]
    pred['labels'] = pred['labels'][bool_mask]
    pred['scores'] = pred['scores'][bool_mask]

    print("Prediction")
    plot_image(img, pred)
    print("Target")
    plot_image(img, anno)

In [7]:
def plot_image(img_tensor, annotation):
    
    fig,ax = plt.subplots(1)
    img = img_tensor.cpu().data
    colours = [None,'r','g','y']

    # Display the image
    ax.imshow(img.permute(1, 2, 0))
    
    i = 0
    for box in annotation["boxes"]:
        
        xmin, ymin, xmax, ymax = box
        col = colours[annotation["labels"][i]]

        # Create a Rectangle patch
        rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=2,edgecolor=col,facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)
        i += 1

    plt.show()

In [8]:
def on_data_change(change):
    lbl_pred.value = ''
    img = TensorImage(image2tensor(PILImage.create(btn_upload.data[-1]))).float()/255 #PILImage.create(btn_upload.data[-1]).ToTensor()
    img.unsqueeze_(0)
    out_pl.clear_output()
    #with out_pl: display(img.to_thumb(256,256))
    pred = model(img)[0] 
     
    pred = remove_dupe_boxes(pred, 0.75)
    bool_mask = pred['scores'] > 0.5
    pred['boxes'] = pred['boxes'][bool_mask]
    pred['labels'] = pred['labels'][bool_mask]
    pred['scores'] = pred['scores'][bool_mask]
    
    #lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
    with out_pl: plot_image(img[0] , pred)

In [9]:
btn_upload.observe(on_data_change, names=['data'])

In [10]:
display(VBox([widgets.Label('Upload an image of people with/without face masks'), btn_upload, out_pl, lbl_pred]))